In [1]:
import os
import sys
import cv2
import csv
import locale
import sys
import rasterio
import numpy as np
import pandas as pd

from scipy.stats import mode
from PIL import Image
from rasterio.transform import from_origin

Image.MAX_IMAGE_PIXELS = 933120000
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
import os
ScoreThreshold = 0.5
CropSize = 300
model_type = "vit_h"
numberPoints = 32

bandOrder = "_BGR_"

parametresName = "S"+str(int(ScoreThreshold*100))+"_A"+str(CropSize)+bandOrder+model_type+"_PPS"+str(numberPoints)

if os.path.isdir('objekty/'+parametresName):
  print("existuje")
else:
  os.makedirs('objekty/'+parametresName)

print(parametresName)

if bandOrder == "_RGB_":
  ImageRGB = np.array(cv2.imread('CO_RGB_8kmExt_8BIT.tif', cv2.COLOR_BGR2RGB),dtype=np.uint8)

if bandOrder == "_BGR_":
  ImageRGB = np.array(cv2.imread('CO_RGB_8kmExt_8BIT.tif'),dtype=np.uint8)

#Pole ze ktereho se zjisti zda je tam neco jineho ,np.dtype = np.uint8nez null
converted_array = np.bool_(np.where(ImageRGB[:,:,0] == 0,0,1))

#delky poli
x_length = ImageRGB.shape[0]
y_length = ImageRGB.shape[1]

#Vysledny vysledek
Multiband = (np.zeros((x_length, y_length),dtype=np.uint8))

step = int(CropSize/1)

ScoreMatice = np.zeros((x_length, y_length),dtype = np.uint8)
IOUMatice = np.ones((x_length, y_length),dtype = np.uint8)



#from segment_anything_hq import sam_model_registry, SamAutomaticMaskGenerator
from segment_anything import sam_model_registry,SamAutomaticMaskGenerator

if model_type == "vit_h":
  sam_checkpoint = "checkpoints/sam_vit_h_4b8939.pth"    # H
if model_type == "vit_b":
  sam_checkpoint = "checkpoints/sam_vit_b_01ec64.pth"  # B

#sam_checkpoint = "/content/drive/MyDrive/JanekProgramovani/Programovani_SAM/checkpoiintModel/sam_hq_vit_h.pth"    # HQ H
#sam_checkpoint = "/content/drive/MyDrive/JanekProgramovani/Programovani_SAM/checkpoiintModel/sam_hq_vit_b.pth"    # HQ B
#sam_checkpoint = "/content/drive/MyDrive/JanekProgramovani/Programovani_SAM/checkpoiintModel/sam_hq_vit_tiny.pth"    # HQ tiny




sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device="cuda")
mask_generator = SamAutomaticMaskGenerator(sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device="cuda"),points_per_side= numberPoints ,stability_score_thresh=ScoreThreshold, pred_iou_thresh=0.5)



S50_A300_BGR_vit_h_PPS32


In [5]:
from datetime import datetime

# Získání aktuálního času
cas = datetime.now()

# Formátování času
format_casu = cas.strftime("%A %H:%M")





if CropSize == 200:
  pocet = 7433
  maxSum = 35000

if CropSize == 300:
  pocet = 3270 # při A300
  maxSum = 70000

if CropSize == 400:
  pocet = 1826 # při A300
  maxSum = 140000

if CropSize == 500:
  pocet = 1100 # při A300
  maxSum = 160000
    
    
if CropSize == 600:
  pocet = 900
  maxSum = 250000
    
if CropSize == 700:
  pocet = 700
  maxSum = 320000

    
if CropSize == 800:
  pocet = 600 # při A300
  maxSum = 400000


j = 0

# startovni pozice moving window
y = 0
while (y) < (y_length):

  x = 0
  while (x) < (x_length):



    if (y+CropSize)>y_length:
      rozmer_y = y_length
    else:
      rozmer_y = y+CropSize


    if (x+CropSize)>x_length:
      rozmer_x = x_length
    else:
      rozmer_x = x+CropSize



    # pokud je v cropu jenom null, tak preskocit
    if np.sum(converted_array[x:rozmer_x, y:rozmer_y]) < maxSum:
      x = x+step
      continue

    j = j + 1


    Crop = ImageRGB[x:rozmer_x, y:rozmer_y, :]#.transpose(1,0,2)





    masks = mask_generator.generate(Crop) # GPU



    # promenna pro iterovani masek
    i = 0

    while i < len(masks):

      a = np.array((masks[i]["segmentation"]),dtype= bool)#.transpose(1,0)
      b = np.uint8(((masks[i]["stability_score"])+0.005)*100)
      c = np.uint8(((masks[i]["predicted_iou"])+0.005)*100)
      d = np.uint16(np.sum(a))
      e = np.array(masks[i]["point_coords"])

      if i == 0:
        Objekty = a
        SS = b
        IOU = c
        area = d
        coords = e
      else:
        Objekty = np.dstack((Objekty, a))
        SS = np.dstack((SS, b))
        IOU = np.dstack((IOU, c))
        area = np.dstack((area, d))
        coords = np.dstack((coords, e))


      i = i+1



    np.savez_compressed('objekty/'+parametresName+'/'+str(x)+"_"+str(y)+'.npz', Objekty=Objekty, SS=SS,IOU= IOU,area= area, coords = coords)
    # prechod na dalsi crop
    print("řádek ",x," ", str(int(j/pocet*100+0.5))+" %")
    x = x+step

  cas = datetime.now()
  print("Sloupec ",y,"  ",str(cas))
  y = y + step


řádek  9000   0 %
řádek  9300   0 %
řádek  9600   0 %
řádek  9900   0 %
řádek  10200   0 %
řádek  10500   0 %
řádek  10800   0 %
řádek  11100   0 %
řádek  11400   0 %
řádek  11700   0 %
řádek  12000   0 %
řádek  12300   0 %
řádek  12600   0 %
řádek  12900   0 %
řádek  13200   0 %
řádek  13500   0 %
Sloupec  0    2024-08-08 09:33:40.132604
řádek  9000   1 %
řádek  9300   1 %
řádek  9600   1 %
řádek  9900   1 %
řádek  10200   1 %
řádek  10500   1 %
řádek  10800   1 %
řádek  11100   1 %
řádek  11400   1 %
řádek  11700   1 %
řádek  12000   1 %
řádek  12300   1 %
řádek  12600   1 %
řádek  12900   1 %
řádek  13200   1 %
řádek  13500   1 %
Sloupec  300    2024-08-08 10:02:41.240004
řádek  8700   1 %
řádek  9000   1 %
řádek  9300   1 %
řádek  9600   1 %
řádek  9900   1 %
řádek  10200   1 %
řádek  10500   1 %
řádek  10800   1 %
řádek  11100   1 %
řádek  11400   1 %
řádek  11700   1 %
řádek  12000   1 %
řádek  12300   1 %
řádek  12600   1 %
řádek  12900   1 %
řádek  13200   1 %
řádek  13500   2 

řádek  11700   12 %
řádek  12000   12 %
řádek  12300   12 %
řádek  12600   12 %
řádek  12900   12 %
řádek  13200   12 %
řádek  13500   12 %
řádek  13800   13 %
řádek  14100   13 %
řádek  14400   13 %
řádek  14700   13 %
řádek  15000   13 %
řádek  15300   13 %
řádek  15600   13 %
řádek  15900   13 %
řádek  16200   13 %
řádek  16500   13 %
řádek  16800   13 %
řádek  17100   13 %
řádek  17400   13 %
řádek  17700   13 %
Sloupec  4500    2024-08-08 21:23:43.277489
řádek  6300   13 %
řádek  6600   13 %
řádek  6900   13 %
řádek  7200   13 %
řádek  7500   13 %
řádek  7800   13 %
řádek  8100   13 %
řádek  8400   13 %
řádek  8700   13 %
řádek  9000   13 %
řádek  9300   13 %
řádek  9600   13 %
řádek  9900   13 %
řádek  10200   13 %
řádek  10500   13 %
řádek  10800   13 %
řádek  11100   13 %
řádek  11400   13 %
řádek  11700   13 %
řádek  12000   14 %
řádek  12300   14 %
řádek  12600   14 %
řádek  12900   14 %
řádek  13200   14 %
řádek  13500   14 %
řádek  13800   14 %
řádek  14100   14 %
řádek  14

řádek  13200   24 %
řádek  13500   24 %
řádek  13800   24 %
řádek  14100   24 %
řádek  14400   25 %
řádek  14700   25 %
řádek  15000   25 %
řádek  15300   25 %
řádek  15600   25 %
řádek  15900   25 %
řádek  16200   25 %
řádek  16500   25 %
řádek  16800   25 %
řádek  17100   25 %
řádek  17400   25 %
řádek  17700   25 %
řádek  18000   25 %
řádek  18300   25 %
řádek  18600   25 %
řádek  18900   25 %
Sloupec  7500    2024-08-09 08:49:33.480231
řádek  7200   25 %
řádek  7500   25 %
řádek  7800   25 %
řádek  8100   25 %
řádek  8400   25 %
řádek  8700   25 %
řádek  9000   25 %
řádek  9300   25 %
řádek  9600   25 %
řádek  9900   25 %
řádek  10200   25 %
řádek  10500   25 %
řádek  10800   25 %
řádek  11100   25 %
řádek  11400   25 %
řádek  11700   25 %
řádek  12000   25 %
řádek  12300   26 %
řádek  12600   26 %
řádek  12900   26 %
řádek  13200   26 %
řádek  13500   26 %
řádek  13800   26 %
řádek  14100   26 %
řádek  14400   26 %
řádek  14700   26 %
řádek  15000   26 %
řádek  15300   26 %
řádek 

řádek  12600   36 %
řádek  12900   36 %
řádek  13200   36 %
řádek  13500   37 %
řádek  13800   37 %
řádek  14100   37 %
řádek  14400   37 %
řádek  14700   37 %
řádek  15000   37 %
řádek  15300   37 %
řádek  15600   37 %
řádek  15900   37 %
řádek  16200   37 %
řádek  16500   37 %
řádek  16800   37 %
řádek  17100   37 %
řádek  17400   37 %
řádek  17700   37 %
řádek  18000   37 %
řádek  18300   37 %
řádek  18600   37 %
řádek  18900   37 %
řádek  19200   37 %
řádek  19500   37 %
Sloupec  10500    2024-08-09 22:23:54.541324
řádek  8700   37 %
řádek  9000   37 %
řádek  9300   37 %
řádek  9600   37 %
řádek  9900   37 %
řádek  10200   37 %
řádek  10500   37 %
řádek  10800   37 %
řádek  11100   37 %
řádek  11400   37 %
řádek  11700   37 %
řádek  12000   37 %
řádek  12300   38 %
řádek  12600   38 %
řádek  12900   38 %
řádek  13200   38 %
řádek  13500   38 %
řádek  13800   38 %
řádek  14100   38 %
řádek  14400   38 %
řádek  14700   38 %
řádek  15000   38 %
řádek  15300   38 %
řádek  15600   38 %


řádek  16800   49 %
řádek  17100   49 %
řádek  17400   49 %
řádek  17700   49 %
řádek  18000   49 %
řádek  18300   49 %
řádek  18600   49 %
řádek  18900   49 %
řádek  19200   49 %
řádek  19500   49 %
Sloupec  13200    2024-08-10 13:47:09.541891
řádek  0   49 %
řádek  300   49 %
řádek  600   49 %
řádek  900   49 %
řádek  2100   49 %
řádek  2400   49 %
řádek  2700   49 %
řádek  3000   49 %
řádek  3300   49 %
řádek  3600   49 %
řádek  3900   49 %
řádek  4200   49 %
řádek  4500   49 %
řádek  4800   49 %
řádek  5100   49 %
řádek  5400   49 %
řádek  5700   49 %
řádek  6000   49 %
řádek  6300   49 %
řádek  6600   49 %
řádek  6900   49 %
řádek  7200   49 %
řádek  7500   50 %
řádek  7800   50 %
řádek  8100   50 %
řádek  8400   50 %
řádek  8700   50 %
řádek  9000   50 %
řádek  9300   50 %
řádek  9600   50 %
řádek  9900   50 %
řádek  10200   50 %
řádek  10500   50 %
řádek  10800   50 %
řádek  11100   50 %
řádek  11400   50 %
řádek  11700   50 %
řádek  12000   50 %
řádek  12300   50 %
řádek  12600

řádek  18900   61 %
řádek  19200   61 %
řádek  19500   61 %
řádek  19800   61 %
řádek  20100   61 %
Sloupec  15000    2024-08-11 06:11:12.650163
řádek  0   61 %
řádek  300   61 %
řádek  600   61 %
řádek  900   61 %
řádek  1200   61 %
řádek  1500   61 %
řádek  1800   61 %
řádek  2100   61 %
řádek  2400   61 %
řádek  2700   61 %
řádek  3000   62 %
řádek  3300   62 %
řádek  3600   62 %
řádek  3900   62 %
řádek  4200   62 %
řádek  4500   62 %
řádek  4800   62 %
řádek  5100   62 %
řádek  5400   62 %
řádek  5700   62 %
řádek  6000   62 %
řádek  6300   62 %
řádek  6600   62 %
řádek  6900   62 %
řádek  7200   62 %
řádek  7500   62 %
řádek  7800   62 %
řádek  8100   62 %
řádek  8400   62 %
řádek  8700   62 %
řádek  9000   62 %
řádek  9300   62 %
řádek  9600   62 %
řádek  9900   62 %
řádek  10200   62 %
řádek  10500   62 %
řádek  10800   62 %
řádek  11100   62 %
řádek  11400   62 %
řádek  11700   62 %
řádek  12000   62 %
řádek  12300   62 %
řádek  12600   62 %
řádek  12900   63 %
řádek  13200   

řádek  3600   74 %
řádek  3900   74 %
řádek  4200   74 %
řádek  4500   74 %
řádek  5100   74 %
řádek  5400   74 %
řádek  5700   74 %
řádek  6000   74 %
řádek  6300   74 %
řádek  6600   74 %
řádek  6900   74 %
řádek  7200   74 %
řádek  7500   74 %
řádek  7800   74 %
řádek  8100   74 %
řádek  8400   74 %
řádek  8700   74 %
řádek  9000   74 %
řádek  9300   74 %
řádek  9600   74 %
řádek  9900   74 %
řádek  10200   74 %
řádek  10500   74 %
řádek  10800   74 %
řádek  11100   74 %
řádek  11400   74 %
řádek  11700   74 %
řádek  12000   74 %
řádek  12300   74 %
řádek  12600   74 %
řádek  12900   74 %
řádek  13200   74 %
řádek  13500   75 %
řádek  13800   75 %
řádek  14100   75 %
řádek  14400   75 %
řádek  14700   75 %
řádek  15000   75 %
řádek  15300   75 %
řádek  15600   75 %
řádek  15900   75 %
řádek  16200   75 %
řádek  16500   75 %
řádek  16800   75 %
řádek  17100   75 %
řádek  17400   75 %
řádek  17700   75 %
řádek  18000   75 %
Sloupec  17100    2024-08-11 19:43:08.829967
řádek  600   75 

řádek  10200   86 %
řádek  10500   86 %
řádek  10800   86 %
řádek  11100   86 %
řádek  11400   86 %
řádek  11700   86 %
řádek  12000   86 %
řádek  12300   86 %
řádek  12600   86 %
řádek  12900   86 %
řádek  13200   86 %
řádek  13500   86 %
řádek  13800   86 %
řádek  14100   86 %
řádek  14400   86 %
řádek  14700   86 %
řádek  15000   86 %
řádek  15300   86 %
řádek  15600   86 %
řádek  15900   86 %
řádek  16200   86 %
řádek  16500   86 %
řádek  16800   87 %
řádek  17100   87 %
řádek  17400   87 %
řádek  17700   87 %
řádek  18000   87 %
řádek  18300   87 %
Sloupec  19500    2024-08-12 07:04:18.322685
řádek  6900   87 %
řádek  7200   87 %
řádek  7500   87 %
řádek  7800   87 %
řádek  8100   87 %
řádek  8400   87 %
řádek  8700   87 %
řádek  9000   87 %
řádek  9300   87 %
řádek  9600   87 %
řádek  9900   87 %
řádek  10200   87 %
řádek  10500   87 %
řádek  10800   87 %
řádek  11100   87 %
řádek  11400   87 %
řádek  11700   87 %
řádek  12000   87 %
řádek  12300   87 %
řádek  12600   87 %
řádek 

řádek  12000   98 %
řádek  12300   98 %
řádek  12600   98 %
řádek  12900   98 %
řádek  13200   98 %
řádek  13500   98 %
řádek  13800   98 %
řádek  14100   98 %
řádek  14400   98 %
řádek  14700   98 %
řádek  15000   98 %
řádek  15300   98 %
řádek  15600   98 %
řádek  15900   98 %
Sloupec  23100    2024-08-13 05:06:09.927649
řádek  9300   98 %
řádek  9600   98 %
řádek  9900   98 %
řádek  10200   98 %
řádek  10500   98 %
řádek  10800   98 %
řádek  11100   98 %
řádek  11400   98 %
řádek  11700   98 %
řádek  12000   98 %
řádek  12300   98 %
řádek  12600   98 %
řádek  12900   98 %
řádek  13200   99 %
řádek  13500   99 %
řádek  13800   99 %
řádek  14100   99 %
řádek  14400   99 %
řádek  14700   99 %
řádek  15000   99 %
řádek  15300   99 %
řádek  15600   99 %
řádek  15900   99 %
Sloupec  23400    2024-08-13 08:17:42.434486
řádek  9300   99 %
řádek  9600   99 %
řádek  9900   99 %
řádek  10200   99 %
řádek  10500   99 %
řádek  10800   99 %
řádek  11100   99 %
řádek  11400   99 %
řádek  11700   9

In [1]:
import tensorflow as tf
print(tf.__version__)

2.13.1


In [2]:
!pip install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     -------------------- ------------------- 30.7/59.7 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 59.7/59.7 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/50.1 kB ? eta -:--:--
     ---------------------------------------- 50.1/50.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/390.2 MB 3.9 MB/s eta 0:01:40
   ---------------------------------------- 0.4/390.2 MB 4.9 MB/s eta 0:01:20
   ---------------------------------------- 0.6/390.2 MB 5.0 MB/s eta 0:01:18
   ---------------------------------------- 0.9/390.2 MB 5.4 MB/s eta 0:01:12
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-fft 1.3.10 requires mkl, which is not installed.
osmnx 1.9.4 requires numpy<1.27,>=1.20, but you have numpy 2.0.2 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
scipy 1.9.3 requires numpy<1.26.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
